<a href="https://colab.research.google.com/github/varshareddykumbham/Speech-Emotion-Recognition-using-RNN-LSTM/blob/main/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob  
import os  
import librosa  
import numpy as np  
 
from keras.utils import to_categorical
import ntpath

In [ ]:
# function to extract feature
def extract_feature(file_name):   
    X, sample_rate = librosa.load(file_name)  
    stft = np.abs(librosa.stft(X))  
    mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)  
    #mfcc_delta = np.mean(librosa.feature.delta(librosa.feature.mfcc(X, sample_rate, 40).T, axis=0))
    #mfcc_delta2 = np.mean(librosa.feature.delta(librosa.feature.mfcc(X, sample_rate, 40).T, axis=0), order=2)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)  
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)  
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)  
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)  
    return mfcc, chroma, mel, contrast, tonnetz #  mfcc_delta, mfcc_delta2, 
 
# function to parse audio file given main dir and sub dir
def parse_audio_files(parent_dir,sub_dirs,file_ext="*.wav"):
    features, labels = np.empty((0, 193)), np.empty(0)  #273: number of features, ori:193
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            try:
                print('process..', fn)
                mfcc, chroma, mel, contrast,tonnetz  = extract_feature(fn)
            except Exception as e:
                print('cannot open', fn)
                continue
            ext_features = np.hstack([mfcc, chroma, mel, contrast, tonnetz])
            #ext_features = np.hstack([mfcc, mfcc_delta, mfcc_delta2, chroma, mel, contrast,tonnetz ])
            features = np.vstack([features, ext_features])
            filename = ntpath.basename(fn)
            labels = np.append(labels, filename.split('-')[2])  # grab 3rd item
            #labels = np.append(labels, fn.split('/')[2].split('-')[1])
    return np.array(features), np.array(labels, dtype = np.int)
 

In [ ]:
main_dir = '/content/drive/MyDrive/audio_speech_actors_01-24'
sub_dir = os.listdir(main_dir)  
print ("\ncollecting features and labels...")  
print("\nthis will take some time...")  
features, labels = parse_audio_files(main_dir,sub_dir)  
labels_oh = to_categorical(labels)   # one hot conversion from integer to binary
print("done") 


collecting features and labels...

this will take some time...
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-02-01-01-02-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-03-01-01-01-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-01-01-02-01-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-03-01-02-01-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-02-01-02-02-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-01-01-02-02-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-01-01-01-02-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-02-02-02-02-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-02-02-02-01-21.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_21/03-01-02-02-01-01-21.wav
proces

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=1012
  n_fft, y.shape[-1]


process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-03-01-01-02-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-03-02-01-01-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-03-01-02-01-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-04-01-01-01-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-06-02-01-02-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-05-02-02-02-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-05-02-01-02-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-06-02-02-01-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-05-01-01-01-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03-01-06-01-01-01-13.wav
process.. /content/drive/MyDrive/audio_speech_actors_01-24/Actor_13/03

In [ ]:
np.save('X', features)
np.save('y', labels_oh)